## 모델 학습을 위한 데이터 전처리

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from google.colab import drive
import os

drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/ML_Project'
SAVE_PATH_TRAIN = os.path.join(BASE_DIR, 'colored_mnist_train.npz')
SAVE_PATH_TEST = os.path.join(BASE_DIR, 'colored_mnist_test.npz')

NUM_CLASSES_DIGIT = 10  # 숫자 (0~9)
NUM_CLASSES_COLOR = 7   # 색상 (무지개 7색)

# 학습 데이터 로드
train_data = np.load(SAVE_PATH_TRAIN)
X_train = train_data['images']
y_train_digit = train_data['labels_digit']
y_train_fg = train_data['labels_fg']
y_train_bg = train_data['labels_bg']

# 테스트 데이터 로드
test_data = np.load(SAVE_PATH_TEST)
X_test = test_data['images']
y_test_digit = test_data['labels_digit']
y_test_fg = test_data['labels_fg']
y_test_bg = test_data['labels_bg']

print(f"원본 학습 데이터 Shape: {X_train.shape}")
print(f"원본 테스트 데이터 Shape: {X_test.shape}")

Mounted at /content/drive
원본 학습 데이터 Shape: (60000, 28, 28, 3)
원본 테스트 데이터 Shape: (10000, 28, 28, 3)


### 원-핫 인코딩

In [4]:
# 학습 데이터 라벨 변환
y_train_digit_oh = to_categorical(y_train_digit, num_classes=NUM_CLASSES_DIGIT)
y_train_fg_oh = to_categorical(y_train_fg, num_classes=NUM_CLASSES_COLOR)
y_train_bg_oh = to_categorical(y_train_bg, num_classes=NUM_CLASSES_COLOR)

# 테스트 데이터 라벨 변환
y_test_digit_oh = to_categorical(y_test_digit, num_classes=NUM_CLASSES_DIGIT)
y_test_fg_oh = to_categorical(y_test_fg, num_classes=NUM_CLASSES_COLOR)
y_test_bg_oh = to_categorical(y_test_bg, num_classes=NUM_CLASSES_COLOR)

print(f"  - 원본: {y_train_digit[0]}")
print(f"  - 원-핫: {y_train_digit_oh[0]}")
print(f"\n라벨 Shape (Train): {y_train_digit_oh.shape}")

  - 원본: 6
  - 원-핫: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]

라벨 Shape (Train): (60000, 10)


### 검증 데이터 분리

In [3]:
# X_train과 원-핫 인코딩된 라벨 3개를 함께 전달하여 분리
(
    X_train_final, X_val,
    y_train_digit_final, y_val_digit,
    y_train_fg_final, y_val_fg,
    y_train_bg_final, y_val_bg
) = train_test_split(
    X_train,
    y_train_digit_oh,
    y_train_fg_oh,
    y_train_bg_oh,
    test_size=0.2,
    random_state=42
)

print("\n--- 최종 전처리 완료 ---")
print("1. 최종 학습용 데이터 (X_train_final):", X_train_final.shape)
print("2. 검증용 데이터 (X_val):", X_val.shape)
print("3. 테스트용 데이터 (X_test):", X_test.shape)
print("-" * 20)
print("최종 학습용 라벨 (숫자):", y_train_digit_final.shape)
print("검증용 라벨 (숫자):", y_val_digit.shape)
print("테스트용 라벨 (숫자):", y_test_digit_oh.shape)
print("-" * 20)
print("최종 학습용 라벨 (전경):", y_train_fg_final.shape)
print("최종 학습용 라벨 (배경):", y_train_bg_final.shape)


--- 최종 전처리 완료 ---
1. 최종 학습용 데이터 (X_train_final): (48000, 28, 28, 3)
2. 검증용 데이터 (X_val): (12000, 28, 28, 3)
3. 테스트용 데이터 (X_test): (10000, 28, 28, 3)
--------------------
최종 학습용 라벨 (숫자): (48000, 10)
검증용 라벨 (숫자): (12000, 10)
테스트용 라벨 (숫자): (10000, 10)
--------------------
최종 학습용 라벨 (전경): (48000, 7)
최종 학습용 라벨 (배경): (48000, 7)


### 데이터 증강: 회전, 이동, 확대/축소 등

In [5]:
# 데이터 증강을 위한 레이어 정의
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomRotation(0.05), # -18도 ~ +18도 사이에서 랜덤 회전 (360*0.05=18)
  tf.keras.layers.RandomZoom(0.1),      # 10% 내외로 랜덤 확대/축소
  tf.keras.layers.RandomWidth(0.1),     # 10% 내외로 랜덤 가로 이동
  tf.keras.layers.RandomHeight(0.1),    # 10% 내외로 랜덤 세로 이동
])